In [24]:
from scipy.signal import sawtooth
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [28]:
def noteToHz (notes, bpm=120) :
    midi_notes = []
    for note in range(128):
        midi_notes.append(440*pow(2, (note-69)/12))

    let2num = {'c': 0, 'c#': 1, 'db': 1, 'd': 2, 'd#': 3, 'eb': 3, 'e': 4, 'fb': 4, 'f': 5, 'e#': 5, 'F#': 6, 'gb': 6,
               'g': 7, 'g#': 8, 'ab': 8, 'a': 9, 'a#': 10, 'bb': 10, 'b': 11, 'cb': 11} #all possible note inputs
    whole_note = 240 / bpm
    output = []
    for chord in notes:
        #handling the note to frequency conversion first
        if type(chord[0]) == list: #this means it's a chord
            pitch = []
            for note in range(len(chord[0])):
                octave = int(chord[0][note][0]) * 12 + 12
                letter = let2num[chord[0][note][1:]]
                pitch.append(midi_notes[octave+letter])
        else: #it's a single note
            if chord[0] != 'rest':
                octave = int(chord[0][0]) * 12 + 12
                letter = let2num[chord[0][1:]]
                pitch = [midi_notes[octave+letter]]
            else: #it's a rest
                pitch = [0.0]
        
        #handling the length to second conversion second
        dur = chord[1]*whole_note
        output.append([pitch, dur])
    return output

In [127]:
def envelope(dur, a=0, d=0, s=1, r=0, fs=44100):
    import numpy as np
    samps = int(fs*dur)
    attack = np.linspace(0, 1, int(fs*a))
    decay = np.linspace(1, s, int(fs*d))
    if dur < a+d:
        env = np.concatenate((attack, decay))
        release = np.linspace(env[samps], 0, int(fs*r))
        env = np.concatenate((env[:samps], release))
    else:
        sussy = np.full(int((dur - a - d) * fs), s)
        release = np.linspace(s, 0, int(fs*r))
        env = np.concatenate([attack, decay, sussy, release])
    return env

def num_del (audio, ms, mix=0.5, delays=3, fs=44100):
    import numpy as np
    sec = ms/1000.0
    samps = int(delays * sec * fs)
    pad = np.zeros(samps)
    dry = np.concatenate((audio, pad), axis=0)
    wet = np.zeros(dry.size)
    for delay in range(delays):
        wet += np.roll(dry, (delay+1)*int(sec*fs))/(1+delay)
    output = (1-mix)*dry + mix*wet
    return output

In [133]:
def bell_base_generator(alist, ms = 0, delays = 0, attack = 0, decay = 5, sustain = 0, release = 2, mix = 0.5, fs=44100, reverse = False):
    from scipy.signal import sawtooth
    I = 2.234
    da_chord = np.array([])
    dsr = [decay, sustain, release]
    songlength = 0
    
    # Finds the total song length
    for chord in alist:
        songlength += chord[1]
        
    songsamps = int(songlength*fs)
    cur_samp = 0
    song = np.zeros(int(songsamps+fs*dsr[2]))
    
    for chord in alist:
        da_chord = np.zeros(int(fs*(chord[1]+dsr[2])))
        t = np.linspace(0,chord[1]+dsr[2],int(fs*(chord[1]+dsr[2])))
        for note in chord[0]:
            if note == 0:
                da_chord+=np.zeros(int(fs*(chord[1]+dsr[2])))
                break
            mod = I * sawtooth(2*np.pi*note*(2)*(t+1/(note*8)), .5)
            fm = sawtooth(2*np.pi*note*(t+1/(note*4))+mod, .5)
            da_chord += fm
            
        env = da_chord*envelope(chord[1], d = dsr[0], s = dsr[1], r = dsr[2])
        pad = np.concatenate((env, np.zeros(int(songsamps-fs*chord[1]))))
        rolled = np.roll(pad, cur_samp)
        cur_samp += int(chord[1]*fs)
        song += rolled
    delayed = num_del(song, ms, mix=mix, delays=delays, fs=fs) 
    if reverse:
        delayed = delayed[::-1]
    return delayed

In [134]:
example_notes = [[['3b', '3g', '4d'], 0.25], ['3a', 0.25], ['3g', 0.25], ['3a', 0.25],['3b', 0.25], ['3b', 0.25], ['3b', 0.25], ['rest', 0.25]]
out = noteToHz(example_notes, bpm=120)

#alist = [[[300], .5], [[440, 330], .25]]
x = bell_base_generator(out, ms=100, decay = 0.5, delays=10, mix=0.5, reverse = True)
Audio(x, rate = 44100)